## Fresnel Transfer Function Approach

1. Given $\lambda$,z, and the largest width of the aperture $l$, calculate the Fresnel Number $N_f$ appropriate for the entire aperture.

In [3]:
#First, lets define our units
cm = 1e-2
mm = 1e-3
um = 1e-6
nm = 1e-9

In [1]:
N_f = ( (l/2)**2 ) / (wavelength * z)

NameError: name 'l' is not defined